## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有8種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

```
觀察狀態 = 'dry', 'dry', 'dry'
Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104
Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888
###<your answers>###

最大機率為: Sunny, Sunny, Sunny
```

### 根據上述條件, 寫出Viterbi應用程式

In [1]:
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry
states = ('sunny', 'rainy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny': {'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

In [2]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{},{},{}]
    # Initialize base cases (t == 0), states = ('sunny', 'rainy')
    for y in states:
        V[0][y] = start_p[y] * emit_p[y]['dry']

# {'final_state': [1, 'sunny', 'sunny', q, 1, 'sunny', 'rainy', q], 'sunny': q, 'rainy': q}
    # Run Viterbi for t > 0
    for t in range(1,len(obs)): 
        V[t]['final_state'] = []
        for y in states:
            if t == 1:
                q1 = V[0]['sunny'] * trans_p['sunny'][y] * emit_p[y]['dry']
                q2 = V[0]['rainy'] * trans_p['rainy'][y] * emit_p[y]['dry']
                if max(q1,q2)==q1:
                    V[t][y]=q1
                    V[t]['final_state'] = V[t]['final_state']+[t, 'sunny', y, q1]
                    # V[t]['final_state'] = V[t]['final_state']+[t, str(t-1)+'-sunny', str(t)+'-'+y, q1]
                    #final_state = [t, y]
                else:
                    V[t][y]=q2
                    V[t]['final_state'] = V[t]['final_state']+[t, 'rainy', y, q1]
                    # V[t]['final_state'] = V[t]['final_state']+[t, str(t-1)+'-rainy', str(t)+'-'+y, q1]
            else:
                q1 = V[t-1]['sunny'] * trans_p['sunny'][y] * emit_p[y]['dry']
                q2 = V[t-1]['rainy'] * trans_p['rainy'][y] * emit_p[y]['dry']                
                if max(q1,q2)==q1:
                    V[t][y]=q1
                    V[t]['final_state'] = V[t]['final_state']+[t, 'sunny', y, q1]
                    # V[t]['final_state'] = V[t]['final_state']+[t, str(t-1)+'-sunny', str(t)+'-'+y, q1]
                else:
                    V[t][y]=q2
                    V[t]['final_state'] = V[t]['final_state']+[t, 'rainy', y, q1]
                    # V[t]['final_state'] = V[t]['final_state']+[t, str(t-1)+'-rainy', str(t)+'-'+y, q1]
    print(V)

    print()
    print("result: ")

    path = [0 for i in range(len(obs))]
         

    if max(V[len(obs)-1]['sunny'], V[len(obs)-1]['rainy']) == V[len(obs)-1]['sunny']:
        print('最大機率為: {}'.format(V[len(obs)-1]['sunny']))
        path[len(obs)-1] = 'sunny'
        for i in range(len(obs)-2,0,-1):
            path[i]= V[i+1]['final_state'][1]            
    else:
        print('最大機率為: {}'.format(V[len(obs)-1]['rainy']))
        path[len(obs)-1] = 'rainy'
        for i in range(len(obs)-2,0,-1):
            path[i]= V[i+1]['final_state'][1]
    path[0] = V[1]['final_state'][1]

    print(f'所有路徑為: {path}')

In [3]:
result = viterbi(observations,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probatility)

[{'sunny': 0.24, 'rainy': 0.06}, {'final_state': [1, 'sunny', 'sunny', 0.08639999999999999, 1, 'sunny', 'rainy', 0.009600000000000001], 'sunny': 0.08639999999999999, 'rainy': 0.009600000000000001}, {'final_state': [2, 'sunny', 'sunny', 0.031103999999999993, 2, 'sunny', 'rainy', 0.0034560000000000003], 'sunny': 0.031103999999999993, 'rainy': 0.0034560000000000003}]

result: 
最大機率為: 0.031103999999999993
所有路徑為: ['sunny', 'sunny', 'sunny']


## 測試內容
[{'sunny': 0.24, 'rainy': 0.06}, {'final_state': [1, '0-sunny', '1-sunny', 0.08639999999999999, 1, '0-sunny', '1-rainy', 0.009600000000000001], 'sunny': 0.08639999999999999, 'rainy': 0.009600000000000001}, {'final_state': [2, '1-sunny', '2-sunny', 0.031103999999999993, 2, '1-sunny', '2-rainy', 0.0034560000000000003], 'sunny': 0.031103999999999993, 'rainy': 0.0034560000000000003}]